# Feature Extraction using HoG

In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
input0 = './Dataset/Augmented/'
temp = ['Bowlegs', 'Knock-knee', 'Normal Knee']

In [3]:
for j in temp:
    for filename in os.listdir(input0 + j):
        img = cv2.resize(cv2.imread(input0 + j + '/' + filename, 0), (256, 256))
        #initialise HoG descriptor
        cell_size = (32, 32)  # h x w in pixels
        block_size = (2, 2)  # h x w in cells
        nbins = 9  # number of orientation bins


        # winSize is the size of the image cropped to an multiple of the cell size
        # cell_size is the size of the cells of the img patch over which to calculate the histograms
        # block_size is the number of cells which fit in the patch

        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                        img.shape[0] // cell_size[0] * cell_size[0]),
                    _blockSize=(block_size[1] * cell_size[1],
                                block_size[0] * cell_size[0]),
                    _blockStride=(cell_size[1], cell_size[0]),
                    _cellSize=(cell_size[1], cell_size[0]),
                    _nbins=nbins)


        descriptor = hog.compute(img)
        out=pd.DataFrame(descriptor)

        #append to the csv file
        csv_data=out.to_csv('./HoG/HoG_' + j + '.csv', mode='a', header=False, index=False)
        


# K-means

In [27]:
data1 = pd.read_csv('./HoG/HoG_Bowlegs.csv')
data2 = pd.read_csv('./HoG/HoG_Knock-knee.csv')
data3 = pd.read_csv('./HoG/HoG_Normal Knee.csv')

data1 = data1.astype(np.float64)
data2 = data1.astype(np.float64)
data3 = data1.astype(np.float64)

data1.append(data2)
data1.append(data3)

data1

C:\Users\sumit\AppData\Local\Temp\ipykernel_23708\75713987.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1.append(data2)
C:\Users\sumit\AppData\Local\Temp\ipykernel_23708\75713987.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1.append(data3)


,0.046085335
0,0.029243
1,0.019161
2,0.019308
3,0.037278
4,0.040835
...,...
1695198,0.352727
1695199,0.352727
1695200,0.009651
1695201,0.001085


In [28]:
#performing kmeans on each class
#Bowlegs
kmeans1 = KMeans(n_clusters=5)
kmeans1.fit(data1)

"""#Knock Knees
kmeans2 = KMeans(n_clusters=5)
kmeans2.fit(data2)

#Normal
kmeans3 = KMeans(n_clusters=5)
kmeans3.fit(data3)"""

'#Knock Knees\nkmeans2 = KMeans(n_clusters=5)\nkmeans2.fit(data2)\n\n#Normal\nkmeans3 = KMeans(n_clusters=5)\nkmeans3.fit(data3)'

In [29]:
hist1=np.histogram(kmeans1.labels_,bins=[0,1,2,3,4,5])
#hist2=np.histogram(kmeans2.labels_,bins=[0,1,2,3,4,5])
#hist3=np.histogram(kmeans3.labels_,bins=[0,1,2,3,4,5])

print('histogram of bowed')
print(hist1,"\n")

"""print('histogram of knocked')
print(hist2,"\n")

print('histogram of normal')
print(hist3,"\n")"""

histogram of bowed
(array([662861, 260517, 396458, 136477, 238890], dtype=int64), array([0, 1, 2, 3, 4, 5])) 



'print(\'histogram of knocked\')\nprint(hist2,"\n")\n\nprint(\'histogram of normal\')\nprint(hist3,"\n")'

In [30]:
#performing kmeans prediction of the entire apple dataset with the pretrained kmeans model
#initialising i=0; as its the first class
i=0
for j in temp:
    data=[]
    for filename in os.listdir(input0 + j):
        path = input0 + j + '/' + filename
        a=cv2.imread(path)
        resize=(256,256)
        img=cv2.resize(a,resize)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        cell_size = (32, 32)  # h x w in pixels
        block_size = (2, 2)  # h x w in cells
        nbins = 9  # number of orientation bins


        # winSize is the size of the image cropped to an multiple of the cell size
        # cell_size is the size of the cells of the img patch over which to calculate the histograms
        # block_size is the number of cells which fit in the patch

        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                        img.shape[0] // cell_size[0] * cell_size[0]),
                    _blockSize=(block_size[1] * cell_size[1],
                                block_size[0] * cell_size[0]),
                    _blockStride=(cell_size[1], cell_size[0]),
                    _cellSize=(cell_size[1], cell_size[0]),
                    _nbins=nbins)


        descriptor = hog.compute(img)
        out=pd.DataFrame(descriptor)

        #drop first coloumn as it's the no of feature detected. Not required.
        #append to the csv file
        
        #predict values of feature vector with pretrained kmeans
        #ValueError: Buffer dtype mismatch, expected 'float' but got 'double', in order to avoid this dtype=np.double
        array_double = np.array(out, dtype=np.double)

        a=kmeans1.predict(array_double)
        hist=np.histogram(a,bins=[0,1,2,3,4,5])
        #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
        data.append(hist[0])
    
    #convert Array to Dataframe and append to the list
    Output = pd.DataFrame(data)
    #add row class 
    Output["Class"] = i 
    csv_data=Output.to_csv('./HoG/HoG_'+ j + 'Final.csv', mode='a', index=False)
    i += 1

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with

In [14]:
final = pd.read_csv('./HoG/HoG_BowlegsFinal.csv')
temp1 = final.append(pd.read_csv('./HoG/HoG_Knock-kneeFinal.csv'))
tc = pd.read_csv('./HoG/HoG_Normal KneeFinal.csv')
tcc = temp1.append(tc)

csv_data = tcc.to_csv('./HoG/HoG_FeatureFinal.csv', mode='a', index=False)
print(tcc)

        0    1    2    3    4  Class
0     714  273  210  179  388      0
1     538  444  221   60  501      0
2     498  386  308   91  481      0
3    1508   25   40   97   94      0
4     702  265  236  179  382      0
..    ...  ...  ...  ...  ...    ...
953   953   57  124  222  408      2
954   694  296  323   80  371      2
955   623  275  272  152  442      2
956   810  193  187  167  407      2
957   835  214  208  175  332      2

[2902 rows x 6 columns]


C:\Users\sumit\AppData\Local\Temp\ipykernel_23708\3165544784.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp1 = final.append(pd.read_csv('./HoG/HoG_Knock-kneeFinal.csv'))
C:\Users\sumit\AppData\Local\Temp\ipykernel_23708\3165544784.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tcc = temp1.append(tc)


# Data spliting

In [15]:
df = pd.read_csv('./HoG/HoG_FeatureFinal.csv')
X_train = df.iloc[:, 0:5]
Y_train = df.iloc[:, 5:6]
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, 
                                                      test_size=0.3, 
                                                      stratify=Y_train, 
                                                      random_state=8)

# KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

knn = KNeighborsClassifier(n_neighbors=15)
  
knn.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=15)

In [17]:
print('\nResults obtained for KNN')

# accuracy on training data
y_pred_t = knn.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = knn.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))



Results obtained for KNN

Results obtained on Training Data
Accuracy on Train data :  0.5745937961595273
Precision:  0.5779726563766076
Recall:  0.574216455302533
F-score:  0.5734704270388503

Results obtained on Testing Data
Accuracy on Test data :  0.42365097588978184
Precision:  0.42791209351076026
Recall:  0.42343063402385434
F-score:  0.4246884866582545


# XgBoost

In [ ]:
import xgboost as xgb

classifier = xgb.XGBClassifier(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

classifier.fit(train_x, train_y)

In [20]:
print('\nResults obtained for XgBoost')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for XgBoost

Results obtained on Training Data
Accuracy on Train data :  0.7341211225997046
Precision:  0.7397758692494488
Recall:  0.7334603396335989
F-score:  0.7330264966670867

Results obtained on Testing Data
Accuracy on Test data :  0.42824339839265213
Precision:  0.4271659118778944
Recall:  0.42762084118016325
F-score:  0.42575935500007517


# SVM

In [21]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

subclassifier = SVC(kernel='rbf')
classifier = OneVsOneClassifier(estimator=subclassifier)
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


OneVsOneClassifier(estimator=SVC())

In [22]:
print('\nResults obtained for SVM')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for SVM

Results obtained on Training Data
Accuracy on Train data :  0.4588872476612506
Precision:  0.4644333180842996
Recall:  0.4573645989617206
F-score:  0.44925922062350593

Results obtained on Testing Data
Accuracy on Test data :  0.4626865671641791
Precision:  0.46282736175410116
Recall:  0.4616290018832392
F-score:  0.4573447763832897


# Naive Bayes

In [25]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [26]:
print('\nResults obtained for Naive Bayes')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for Naive Bayes

Results obtained on Training Data
Accuracy on Train data :  0.44903988183161003
Precision:  0.45006422208982455
Recall:  0.44716492565679844
F-score:  0.4320106201261485

Results obtained on Testing Data
Accuracy on Test data :  0.4443168771526981
Precision:  0.43887098218282744
Recall:  0.44280838041431264
F-score:  0.433148145698753
